## One hot models
include one-hot CNN and one-hot Residual models

In [1]:
import tensorflow as tf
import h5py 
import scipy.stats
import numpy as np
import os
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
celltype_list = ['K562','HepG2']

2024-01-17 16:20:04.543903: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
tf.get_logger().setLevel('ERROR')
for celltype in celltype_list:
    cnn_dir = '../model/lenti_MPRA/lenti_MPRA_onehot/'+celltype+'/base_CNN_model.h5'
    rb_dir = '../model/lenti_MPRA/lenti_MPRA_onehot/'+celltype+'/ResNet.h5'
    print('###########' + celltype + '########')
    exp_df = pd.read_csv('../data/CAGI/'+celltype+'/metadata.csv')
    target = exp_df['6'].values.tolist()
    file = h5py.File("../data/CAGI/"+celltype+"/onehot.h5", "r")
    alt = file['alt']
    ref = file['ref']
    
    for model_dir in [cnn_dir,rb_dir]:
        print(model_dir.split('/')[-1])
        model = tf.keras.models.load_model(model_dir)
        alt_pred = model.predict(alt,verbose=0)
        ref_pred = model.predict(ref,verbose = 0)
        pred = alt_pred - ref_pred
        perf = {}
        for exp in exp_df['8'].unique():
            sub_df = exp_df[exp_df['8'] == exp]
            exp_target = np.array(target)[sub_df.index.to_list()]
            exp_pred = np.squeeze(pred)[sub_df.index.to_list()]
            perf[exp] = scipy.stats.pearsonr(exp_pred,exp_target)[0]
        print(perf)
        print(np.mean(list(perf.values())))


###########K562########
base_CNN_model.h5


/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2024-01-17 16:30:10.468787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-17 16:30:10.574442: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8800


{'PKLR': 0.42567543950445913}
0.42567543950445913
ResNet.h5
{'PKLR': 0.5509142235337524}
0.5509142235337524
###########HepG2########
base_CNN_model.h5


/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


{'LDLR': 0.3140119684297847, 'SORT1': 0.26455394176081337, 'F9': 0.3597829131304178}
0.3127829411070053
ResNet.h5
{'LDLR': 0.5142895208051962, 'SORT1': 0.4411468053560604, 'F9': 0.5025298928547497}
0.4859887396720021


## Embedding models
GPN embedding trained models

In [1]:
import tensorflow as tf
import h5py 
import scipy.stats
import numpy as np
import os
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
celltype_list = ['K562','HepG2']

2024-01-18 11:44:35.051205: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
for celltype in celltype_list:
    print('######'+celltype+'######')
    #Load model and dataset
    model_dir = '../model/lenti_MPRA/lenti_MPRA_embed/'+celltype+'/gpn.h5'
    data = '../data/CAGI/cagi_512_gpn.h5'
    file = h5py.File(data,'r')
    model = tf.keras.models.load_model(model_dir)
    
    #Read CAGI metadata
    exp_df = pd.read_csv('../data/CAGI/'+celltype+'/metadata.csv')
    target = exp_df['6'].values.tolist()

    #zero-shot predictions
    alt_pred = model.predict(file['alt'],verbose = 0)
    ref_pred = model.predict(file['ref'],verbose=0)
    pred = alt_pred - ref_pred

    #evaluations
    perf = {}
    for exp in exp_df['8'].unique():
        sub_df = exp_df[exp_df['8'] == exp]
        exp_target = np.array(target)[sub_df.index.to_list()]
        exp_pred = np.squeeze(pred)[sub_df.index.to_list()]
        perf[exp] = scipy.stats.pearsonr(exp_pred,exp_target)[0]
    print(perf)
    print(np.mean(list(perf.values())))

######K562######


2024-01-18 11:44:42.142873: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78973 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:07:00.0, compute capability: 8.0
/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


KeyError: "Unable to synchronously open object (object 'alt' doesn't exist)"

In [3]:
file.keys()

<KeysViewHDF5 ['alt', 'ref']>

In [4]:
file.keys()

<KeysViewHDF5 ['llr']>